# EDA i Feature Engineering - część 1
- Wczytanie danych tras karetek z pliku Parquet i ustawienie odpowiedniego układu współrzędnych (CRS) dla analiz przestrzennych.
- Usunięcie tras z brakującą geometrią oraz takich, gdzie wszystkie punkty znajdują się w promieniu 200 m od punktu początkowego (trasy nieprawidłowe lub postojowe).
- Porównanie współrzędnych referencyjnych (Szerokość geograficzna, Dlugość geograficzna) z początkiem i końcem każdej trasy w celu sprawdzenia spójności danych GPS.
- Konwersja i uproszczenie kodów priorytetów (Kod pilności) do wartości globalnych.
- Tworzenie interaktywnych map tras z zaznaczeniem punktu startu, końca i punktu referencyjnego.
- Identyfikacja i usunięcie tras typu round-trip oraz pętli.
- Usunięcie pustych tras (z brakującą geometrią).
- Odfiltrowanie tras na podstawie czasów odjazdu i powrotu, przycięcie punktów trasy do rzeczywistego czasu przejazdu, zamiast filtrowania od czasu zgłoszenia.


In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
import numpy as np
from shapely.geometry import LineString
import folium

data_path = r"X:\dane_karetki\dojazdy_df.parquet"

In [2]:
columns = [
    "Czas wezwania",
    "Czas wyjazdu ZRM",
    "Czas powrotu ZRM",
    "Powód wezwania",
    "Kod pilności",
    "Identyfikator ZRM",
    "Dlugość geograficzna",
    "Szerokość geograficzna",
    "Identyfikator pojazdu",
    "Rodzaj wyjazdu 0- na sygnale, 1 -zwykly",
    "Typ zespolu",
    "Określenie wieku pacjenta 0- dziecko, 1 - dorosly",
    "ID_GPS",
    "Line",
    "Line_time",
    "Line_points",
    "Line_unique_points"
]
# if 'df' not in globals(): # Avoid reading data again

df = pd.read_parquet(data_path, engine='pyarrow', columns=columns)
df = df.sort_values(by='Czas wezwania', ascending=True)

# sample to 5% of the data
# df = df.sample(frac=0.1)
df = df.reset_index(drop=True)


df['Line'] = df['Line'].apply(wkt.loads)
    # df['Line_short'] = df['Line_short'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry='Line')
gdf = gdf.set_crs(4326)
print(df.shape)



(193233, 17)


In [3]:
gdf.head(10)

,Czas wezwania,Czas wyjazdu ZRM,Czas powrotu ZRM,Powód wezwania,Kod pilności,Identyfikator ZRM,Dlugość geograficzna,Szerokość geograficzna,Identyfikator pojazdu,"Rodzaj wyjazdu 0- na sygnale, 1 -zwykly",Typ zespolu,"Określenie wieku pacjenta 0- dziecko, 1 - dorosly",ID_GPS,Line,Line_time,Line_points,Line_unique_points
0,2021-03-31 22:11:24,2021-03-31 22:26:13,2021-03-31 22:59:00,Inne,K01-D02,K01-D02,19.848080,50.086277,KR 4F001,1,P,1,181375,"LINESTRING (19.92509 50.07631, 19.92509 50.076...","[2021-03-31 22:11:11, 2021-03-31 22:11:26, 202...",205,32
1,2021-03-31 22:29:09,2021-03-31 22:41:55,2021-04-01 00:37:00,Problemy kardiologiczne,K01 47,K01 47,19.427128,50.096901,KCH CW88,0,S,1,250074,"LINESTRING (19.42848 50.13612, 19.42842 50.136...","[2021-03-31 22:28:33, 2021-03-31 22:30:34, 202...",22,18
2,2021-03-31 22:33:51,2021-03-31 22:34:41,2021-03-31 23:19:00,Zasłabnięcie,K02 68,K02 68,20.947538,49.934174,KT 1925G,1,P,1,158507,"LINESTRING (20.99144 50.01636, 20.99144 50.016...","[2021-03-31 22:33:44, 2021-03-31 22:33:59, 202...",217,47
3,2021-03-31 22:34:53,2021-03-31 22:37:25,2021-04-01 00:05:08,Duszność,K01 062,K01 062,19.803841,49.516422,KNT TU09,0,P,0,173168,"LINESTRING (19.89007 49.54469, 19.88862 49.545...","[2021-03-31 22:34:52, 2021-03-31 22:34:57, 202...",370,34
4,2021-03-31 22:38:57,2021-03-31 22:43:46,2021-04-01 00:09:52,Ból brzucha,K01 122,K01 122,19.895836,50.078758,KR 5Y997,1,P,1,250824,"LINESTRING (19.88432 50.08181, 19.88432 50.081...","[2021-03-31 22:38:48, 2021-03-31 22:39:03, 202...",391,65
5,2021-03-31 22:43:41,2021-03-31 22:44:32,2021-03-31 23:51:00,"Paraliż, bełkotliwa mowa",K02-D02,K02-D02,20.932972,49.998936,KT 1810C,1,P,1,171719,"LINESTRING (20.99177 50.01623, 20.99177 50.016...","[2021-03-31 22:43:37, 2021-03-31 22:43:52, 202...",272,31
6,2021-03-31 22:50:12,2021-03-31 22:55:15,2021-04-01 00:30:46,Problemy kardiologiczne,K01 042,K01 042,20.033554,50.072250,KR 5ST66,0,P,1,74302,"LINESTRING (20.01232 50.09422, 20.01361 50.093...","[2021-03-31 22:50:07, 2021-03-31 22:50:15, 202...",410,16
7,2021-03-31 22:51:59,2021-03-31 22:53:34,2021-03-31 23:07:34,Leży,K01 43,K01 43,19.566053,50.279728,KR 6MY69,0,S,1,410286,"LINESTRING (19.5614 50.26539, 19.5614 50.26539...","[2021-03-31 22:51:53, 2021-03-31 22:52:53, 202...",54,40
8,2021-03-31 22:54:20,2021-03-31 22:55:48,2021-04-01 00:46:00,Zaburzenia psychiczne,K02 86,K02 86,20.720327,49.460987,KN 92599,1,P,1,168320,"LINESTRING (20.63207 49.5624, 20.6321 49.56239...","[2021-03-31 22:54:12, 2021-03-31 22:54:27, 202...",432,290
9,2021-03-31 23:00:05,2021-03-31 23:00:49,2021-04-01 00:31:00,Inne / złe samopoczucie,K02 28,K02 28,20.386808,49.711361,KLI 69697,1,P,1,170159,"LINESTRING (20.41265 49.71624, 20.41265 49.716...","[2021-03-31 22:59:58, 2021-03-31 23:00:13, 202...",365,53


In [4]:
gdf.dtypes

Czas wezwania                                        datetime64[ns]
Czas wyjazdu ZRM                                     datetime64[ns]
Czas powrotu ZRM                                     datetime64[ns]
Powód wezwania                                             category
Kod pilności                                               category
Identyfikator ZRM                                          category
Dlugość geograficzna                                        float32
Szerokość geograficzna                                      float32
Identyfikator pojazdu                                      category
Rodzaj wyjazdu 0- na sygnale, 1 -zwykly                        int8
Typ zespolu                                                category
Określenie wieku pacjenta 0- dziecko, 1 - dorosly              int8
ID_GPS                                                        int64
Line                                                       geometry
Line_time                                       

### Konwersja kodów priorytetów

In [5]:
# policz dla jakiego % wierszy wartości kolumn Identyfikator ZRM i Kod pilności są identyczne
gdf['Identyfikator ZRM'] = gdf['Identyfikator ZRM'].astype(str)
gdf['Kod pilności'] = gdf['Kod pilności'].astype(str)
float((gdf['Identyfikator ZRM'] == gdf['Kod pilności']).mean().round(4) * 100)

100.0

In [6]:
gdf = gdf.drop(["Identyfikator ZRM"], axis=1)

In [7]:
unique_combinations = gdf['Kod pilności'].str[:3].nunique()
unique_combinations

2

Czyli zgodnie z informacjami z internetu (https://www.nik.gov.pl/najnowsze-informacje-o-wynikach-kontroli/przemyskie-pogotowie-gotowe-do-pomocy-ale-z-opoznieniem.html) mamy dwa globalne kody pilności - 1 i 2.

Pozostałe znaki w tej kolumnie  mogą odnosić się do konkretnych zespołów ratownictwa medycznego lub ich lokalizacji, ale bez dostępu do wewnętrznych systemów dyspozytorskich trudno jest jednoznacznie je zinterpretować, dlatego je usuniemy

In [8]:
gdf['Kod pilności'] = gdf['Kod pilności'].str[2].astype(np.int8)
gdf['Kod pilności']

0         1
1         1
2         2
3         1
4         1
         ..
193228    2
193229    1
193230    1
193231    1
193232    2
Name: Kod pilności, Length: 193233, dtype: int8

In [9]:
gdf.dtypes

Czas wezwania                                        datetime64[ns]
Czas wyjazdu ZRM                                     datetime64[ns]
Czas powrotu ZRM                                     datetime64[ns]
Powód wezwania                                             category
Kod pilności                                                   int8
Dlugość geograficzna                                        float32
Szerokość geograficzna                                      float32
Identyfikator pojazdu                                      category
Rodzaj wyjazdu 0- na sygnale, 1 -zwykly                        int8
Typ zespolu                                                category
Określenie wieku pacjenta 0- dziecko, 1 - dorosly              int8
ID_GPS                                                        int64
Line                                                       geometry
Line_time                                                    object
Line_points                                     

## Wykresy tras

In [10]:
import branca.colormap as cm

def plot_route(row):
    row = row.to_crs(4326)
    coords = [(y, x) for x, y in row.iloc[0]['Line'].coords]

    # Create a color map
    colormap = cm.linear.PuRd_09.scale(0, len(coords) - 2)

    # Create map
    m = folium.Map(location=coords[0], zoom_start=13)

    # Draw colored segments
    for i in range(len(coords) - 1):
        segment = [coords[i], coords[i + 1]]
        color = colormap(i)
        folium.PolyLine(segment, color=color, weight=5).add_to(m)

    # Mark the first point (green)
    folium.Marker(
        location=coords[0],
        popup='Start',
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)

    # Mark the last point (red)
    folium.Marker(
        location=coords[-1],
        popup='End',
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(m)

    # Add extra marker from row columns
    lat = row.iloc[0]['Szerokość geograficzna']
    lon = row.iloc[0]['Dlugość geograficzna']
    folium.Marker(
        location=[lat, lon],
        popup='Extra point',
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

    # Add colorbar legend
    colormap.caption = 'Order of points'
    colormap.add_to(m)
    return m

# gdf_short = gpd.GeoDataFrame(gdf, geometry='Line_short', crs=2180)
# gdf_short_4326 = gdf_short.sample(1).to_crs(4326)
#
# single_gdf_4326 = gdf_short_4326.to_crs(4326)
#
# display(plot_route(single_gdf_4326))

### Odflitrowanie tras, których żaden punkt nie wychodzi poza 200m od punktu początkowego

In [11]:
gdf_proj = gdf.to_crs(3857)

def all_points_within_m_proj(line, threshold=800):
    coords = np.array(line.coords)
    start = coords[0]
    dists = np.linalg.norm(coords - start, axis=1)
    return np.all(dists <= threshold)

result = gdf_proj['Line'].apply(all_points_within_m_proj)


In [12]:
round(float(result.sum() / len(gdf_proj) * 100), 2)

18.9

17.25% tras ma wszystkie punkty w odległości 500m od startu

In [13]:
filtered_gdf = gdf_proj[result]

In [14]:
display(plot_route(filtered_gdf.sample(1)))

In [15]:
gdf = gdf_proj[~result]

In [16]:
display(plot_route(gdf.sample(1)))

### Sprawdzenie round tripów

In [17]:
def is_round_trip(line, tolerance=0.001):
    coords = np.array(line.coords)
    n = len(coords)
    if n < 4:
        return False  # Too short to be a round trip

    half = n // 2
    first_half = coords[:half]
    second_half = coords[-half:][::-1]  # Reverse the second half

    # Compare each point in first_half to corresponding in reversed second_half
    diffs = np.linalg.norm(first_half - second_half, axis=1)
    return np.all(diffs < tolerance)

In [18]:
gdf = gdf.copy()
is_round_trip = gdf['Line'].apply(is_round_trip)
is_round_trip.mean().round(4) * 100

0.0

In [19]:
# odfiltruj gdf z round tripami
# round_trips = gdf[gdf['is_round_trip']]

In [20]:
# display(plot_route(round_trips.sample(1)))

### Sprawdzenie pętli

In [21]:
def is_loop(line, tolerance=0.001):
    coords = np.array(line.coords)
    if len(coords) < 2:
        return False
    start, end = coords[0], coords[-1]
    return np.linalg.norm(start - end) < tolerance

In [22]:
float(gdf['Line'].apply(is_loop).mean().round(4) * 100)

0.0

## Usunięcie pustych tras

In [23]:
missing_count = gdf['Line'].isnull().sum()
print(f"Number of missing values in 'Line': {missing_count}")

Number of missing values in 'Line': 0


In [24]:
# usunąć puste trasy
gdf = gdf[gdf['Line'].notnull()]

### Odfiltrowanie tras wg czasów odjazdu zamiast czasów wezwania

In [25]:
line_len = gdf['Line'].apply(lambda x: len(x.coords))
line_time_len = gdf['Line_time'].apply(lambda x: len(x) if isinstance(x, (list, np.ndarray)) else 0)
len_match = line_len == line_time_len

# Print summary
print(f"Percentage of rows with matching lengths: {len_match.mean() * 100:.2f}%")

Percentage of rows with matching lengths: 100.00%


In [26]:
gdf['Czas wyjazdu ZRM'] = pd.to_datetime(gdf['Czas wyjazdu ZRM'])
gdf['Czas powrotu ZRM'] = pd.to_datetime(gdf['Czas powrotu ZRM'])

In [27]:
def fast_to_datetime(arr):
    return pd.to_datetime(arr, format='%Y-%m-%d %H:%M:%S')

gdf['Line_time'] = gdf['Line_time'].apply(fast_to_datetime)

In [28]:
def trim_line_and_time(row):
    start = row['Czas wyjazdu ZRM']
    end = row['Czas powrotu ZRM']
    times = row['Line_time']
    mask = (times >= start) & (times <= end)
    trimmed_coords = np.array(row['Line'].coords)[mask]
    return pd.Series({
        'Line': LineString(trimmed_coords) if len(trimmed_coords) > 1 else None,
        'Line_time': list(times[mask].astype(str))
    })

In [29]:
gdf[['Line', 'Line_time']] = gdf.apply(trim_line_and_time, axis=1)

In [30]:
gdf.head()

,Czas wezwania,Czas wyjazdu ZRM,Czas powrotu ZRM,Powód wezwania,Kod pilności,Dlugość geograficzna,Szerokość geograficzna,Identyfikator pojazdu,"Rodzaj wyjazdu 0- na sygnale, 1 -zwykly",Typ zespolu,"Określenie wieku pacjenta 0- dziecko, 1 - dorosly",ID_GPS,Line,Line_time,Line_points,Line_unique_points
0,2021-03-31 22:11:24,2021-03-31 22:26:13,2021-03-31 22:59:00,Inne,1,19.848080,50.086277,KR 4F001,1,P,1,181375,"LINESTRING (2218050.748 6459501.092, 2218050.7...","[2021-03-31 22:26:26, 2021-03-31 22:26:41, 202...",205,32
3,2021-03-31 22:34:53,2021-03-31 22:37:25,2021-04-01 00:05:08,Duszność,1,19.803841,49.516422,KNT TU09,0,P,0,173168,"LINESTRING (2212541.762 6371280.633, 2212541.7...","[2021-03-31 22:37:31, 2021-03-31 22:37:46, 202...",370,34
4,2021-03-31 22:38:57,2021-03-31 22:43:46,2021-04-01 00:09:52,Ból brzucha,1,19.895836,50.078758,KR 5Y997,1,P,1,250824,"LINESTRING (2213512.3 6460455.303, 2213512.3 6...","[2021-03-31 22:43:48, 2021-03-31 22:44:03, 202...",391,65
6,2021-03-31 22:50:12,2021-03-31 22:55:15,2021-04-01 00:30:46,Problemy kardiologiczne,1,20.033554,50.072250,KR 5ST66,0,P,1,74302,"LINESTRING (2228447.458 6462243.584, 2228447.4...","[2021-03-31 22:55:23, 2021-03-31 22:55:38, 202...",410,16
7,2021-03-31 22:51:59,2021-03-31 22:53:34,2021-03-31 23:07:34,Leży,1,19.566053,50.279728,KR 6MY69,0,S,1,410286,"LINESTRING (2177565.239 6492364.604, 2177565.2...","[2021-03-31 22:53:53, 2021-03-31 22:54:53, 202...",54,40


In [31]:
# zapisz gdf do parquet
gdf.to_parquet(r"X:\dane_karetki\karetki.parquet", engine='pyarrow', index=False)